# SGV 12 Dates & Keywords

A notebook to create a data sets of images from the collection Ernst Brunner that contain dates and keywords. 

In [1]:
import os
import json
from collections import defaultdict

# path to the complete, local, collection data
input_dir = "/Users/maxfrischknecht/mfvk_cloud/A_PhD/07_Data/SGV_12N_Metadata_full"

In [3]:
# loop over all the files in the input_dir
result = []
for file_name in os.listdir(input_dir):
    if file_name.endswith('.json'):
        with open(os.path.join(input_dir, file_name), 'r') as f:
            # the file is open
            data = json.load(f)
            # place to save the extracted data
            reduced_data = []

            for obj in data:
                
                # test if image has id and keywords
                if 'schema:identifier' in obj and 'schema:about' in obj and 'schema:temporal' in obj:
                    reduced_obj = {}

                    # save the filename as id
                    reduced_obj['schema:identifier'] = obj['schema:identifier'][0]['@value']

                    # loop over the array of dates
                    dates_list = []
                    for item in obj['schema:temporal']:
                        if '@value' in item:
                            dates_list.append(item['@value'])
                    reduced_obj["schema:temporal"] = dates_list;

                    # loop over the array of keywords
                    about_list = []
                    for item in obj['schema:about']:
                        if 'display_title' in item:
                            about_list.append(item['display_title'])
                    reduced_obj["schema:about"] = about_list;
 
                    # save the new objects with its properties in a list
                    # print(reduced_obj)
                    reduced_data.append(reduced_obj)

            # save the list of reduced objects in a global list
            result.extend(reduced_data)

In [4]:
print(result[5])
print(len(result))

{'schema:identifier': 'SGV_12N_07936', 'schema:temporal': ['1942'], 'schema:about': ['Heuen', 'Mähen', 'Sense (Werkzeug)', 'Feldarbeit', 'Wiese', 'Steilhang', 'Mann', 'Gras', 'Dorf', 'Landwirtschaft', 'Valorisierung: Museum Burgrain']}
19014


Aggregate all objects together that have the same comment and same date to reduce file size

In [5]:
# Aggregating items
aggregated_result = defaultdict(lambda: {'schema:identifier': []})

for item in result:
    key = (tuple(item['schema:about']), tuple(item['schema:temporal']))
    aggregated_result[key]['schema:identifier'].append(item['schema:identifier'])
    aggregated_result[key]['schema:about'] = item['schema:about']
    aggregated_result[key]['schema:temporal'] = item['schema:temporal']

# Convert aggregated_result to a list
final_result = list(aggregated_result.values())
print(len(final_result))
print(final_result[6])

6008
{'schema:identifier': ['SGV_12N_08450', 'SGV_12N_08495'], 'schema:about': ['Projekt Staudämme'], 'schema:temporal': ['1942']}


In [6]:
with open('./export/sgv-12_dates_keywords.json', 'w') as json_file:
    json.dump(final_result, json_file, indent=4)

print("Data has been exported")

Data has been exported
